## Importing required packages

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## Loading csv file

In [ ]:
from google.colab import files
f=files.upload()

Saving creative_writing2.csv to creative_writing2.csv


In [ ]:
df = pd.read_csv('creative_writing2.csv')

## Dropping Null values

In [ ]:
df.dropna(inplace=True)

## Creating the corpus

In [ ]:
corpus = [(prompt, creative_writing) for prompt, creative_writing in zip(df['prompt'], df['creative_writing'])]

## Fitting the tokenizer in the corpus

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([row[1] for row in corpus])
total_words = len(tokenizer.word_index) + 1


## Creating Input Sequences for Neural Network Training

In [ ]:
input_sequences = []
for _, creative_writing in corpus:
    token_list = tokenizer.texts_to_sequences([creative_writing])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)


## Padding Input Sequences for Uniform Length

In [ ]:
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

## Splitting Input and Target Sequences

In [ ]:
X, y = input_sequences[:,:-1],input_sequences[:,-1]


## Training a Bidirectional LSTM Model for Text Generation

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 50, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
199/199 [==============================] - 26s 107ms/step - loss: 6.3525 - accuracy: 0.0943
Epoch 2/100
199/199 [==============================] - 22s 110ms/step - loss: 5.7149 - accuracy: 0.1322
Epoch 3/100
199/199 [==============================] - 22s 112ms/step - loss: 5.4187 - accuracy: 0.1692
Epoch 4/100
199/199 [==============================] - 22s 110ms/step - loss: 5.1084 - accuracy: 0.1940
Epoch 5/100
199/199 [==============================] - 22s 111ms/step - loss: 4.7948 - accuracy: 0.2088
Epoch 6/100
199/199 [==============================] - 22s 108ms/step - loss: 4.5023 - accuracy: 0.2257
Epoch 7/100
199/199 [==============================] - 22s 108ms/step - loss: 4.2300 - accuracy: 0.2425
Epoch 8/100
199/199 [==============================] - 21s 104ms/step - loss: 3.9803 - accuracy: 0.2593
Epoch 9/100
199/199 [==============================] - 22s 108ms/step - loss: 3.7418 - accuracy: 0.2758
Epoch 10/100
199/199 [==============================] - 21s 108m

## Function for Generating text with a Trained Language Model

In [ ]:
def generate_text(prompt, model, tokenizer, max_sequence_len, num_words=50):
    seed_text = prompt.lower()
    for _ in range(num_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)[0]
        predicted_index = np.argmax(predicted_probs)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break
        seed_text += " " + output_word
    print(seed_text.capitalize())

## Interactive Text Generation with Trained Model

In [ ]:
while True:
    prompt = input("Enter a prompt for writing (type 'exit' to quit):\n")
    if prompt.lower() == 'exit':
        print("Goodbye!")
        break
    print("Here's your writing:")
    generate_text(prompt, model, tokenizer, max_sequence_len)
    print("\n")

Enter a prompt for writing (type 'exit' to quit):
A beautiful sunset
Here's your writing:
A beautiful sunset sunset paint the sky in shades of orange and gold with clouds ablaze and the horizon ablaze with color i felt a sense of gratitude for the beauty of the world and the gift of another day day day patterns of another day day day day day day day day


Enter a prompt for writing (type 'exit' to quit):
About an adventurous journey
Here's your writing:
About an adventurous journey through the shimmering portal i found myself transported to a world of wonder and enchantment twisting where me and fantasy intertwined reality i had entered a world untouched by man i had been transported to a paradise on earth wonder and wonder to the embrace of nature and and possible


Enter a prompt for writing (type 'exit' to quit):
Exploration
Here's your writing:
Exploration the tranquility of the forest i embarked on a peaceful journey guided by the whispers of the wind and lurked in the rhythm of t